# Assignment 1: k-Nearest Neighbors

Fill in your name and student ID here.
- Name: Lin Myat
- Student ID: A0271863X

## Overview

In this assignment, we'll implement KNN step-by-step:

1. **Euclidean Distance:** Measure similarity between data points.
2. **Find Neighbors:** Select the 'K' closest points to a query point.
3. **Predict (Classification and Regression):** Use majority vote from neighbors and average of neighbors' values.
4. **Build KNN Class:** Combine everything into a reusable class.
5. **Practical**: Train a KNN classifier on the training dataset using scikit-learn.

By the end, you'll understand how KNN works and how it makes predictions based on distance. Let’s dive in!

## Instructions

3. **IMPORTANT**: Make sure that all of the cells are runnable and can compile without exception, even if the answer is incorrect. This will significantly help us in grading your solutions.
3. For task 1 to 4, you are only allowed to use basic Python functions in your code (no `NumPy` or its equivalents), unless otherwise stated. You may reuse any functions you have defined earlier. If you are unsure whether a particular function is allowed, feel free to ask any of the TAs.
4. For task 5, you may use the `scikit-learn` library.
5. Your solutions will be evaluated against a set of hidden test cases to prevent hardcoding of the answer. You may assume that the test cases are always valid and does not require exception or input mismatch handling. Partial marks may be given for partially correct solutions


## Task 1 - Compute Euclidean Distance [1 Point]

To find the nearest neighbors, we first need a distance measure to determine how **close** two data points are. 

A possible distance measure is **Euclidean distance**—the straight-line distance between points $\mathbf{p} = [p_1, p_2, ..., p_m]$ and $\mathbf{q} = [q_1, q_2, ..., q_m]$ in $m$ dimensions:

$$
d_\text{euclidean}(\mathbf{p}, \mathbf{q}) = \sqrt{\sum_{i=1}^{m} (p_i - q_i)^2}
$$

Implement the function calculating euclidean distance using `math.sqrt()`, without the use of `numpy`.

In [1]:
# TASK 1
import math 

def euclidean_distance(vec_p, vec_q):
    """
    TODO: Write a function to compute distance between two vectors.
    Call the math.sqrt() function to compute the square root of the sum of squared differences.

    Args:
        vec_p: List or tuple p, with a list of m numbers
        vec_q: List or tuple q, with a list of m numbers

    Returns:
        A float representing the Euclidean distance between the two vectors
    """

    if len(vec_p) != len(vec_q):
        raise ValueError("vec_p and vec_q must have the same length")
    return math.sqrt(sum((xi - yi) ** 2 for xi, yi in zip(vec_p, vec_q)))

# TESTCASES 1
assert math.isclose(euclidean_distance([1, 2, 3], [4, 5, 6]), 5.196152422706632, rel_tol=1e-5)
assert math.isclose(euclidean_distance([5.5, 5.5], [5.5, 5.5]), 0.0, rel_tol=1e-5)
assert math.isclose(euclidean_distance([-6], [-6]), 0.0, rel_tol=1e-5)
print('All test cases passed!') 

All test cases passed!


## Task 2 - Get the K Nearest Neighbors [1 Point]

Now that we can measure distance, let’s use that to find the ```k``` closest training points to a given test point. If there are multiple points with the same distance, we keep the ones that appear first in the training data.

In [2]:
# TASK 2
def get_k_nearest_neighbors(training_data, test_point, k):
    """
    TODO: Return the k nearest neighbors to the test_point.

    Args:
        training_data: list of tuples [(feature_vector, label), ...]
        test_point: list of numbers (the point we're classifying)
        k: number of neighbors to consider
        
    Returns:
        list of labels of the k nearest neighbors in the correct order
    """

    k_neighbors = []
    
    """ YOUR CODE STARTS HERE """

    # compute the distance from the test point to each training example
    distances = []
    for (features, label) in training_data:
        dist = euclidean_distance(features, test_point)
        distances.append((dist, label))

    # sort by distance and return the labels of the k nearest neighbors
    distances.sort(key=lambda x: x[0])
    k_neighbors = [label for (_, label) in distances[:k]]
    """ YOUR CODE ENDS HERE """

    return k_neighbors

# TESTCASES 2.1
training_data = [
    ([1, 2], 'A'),
    ([2, 3], 'B'),
    ([3, 4], 'A'),
    ([5, 5], 'B')
]

assert get_k_nearest_neighbors(training_data, [1.5, 2.5], k=2) == ['A', 'B']
assert get_k_nearest_neighbors(training_data, [4, 4], k=1) == ['A']
assert get_k_nearest_neighbors(training_data, [0, 0], k=3) == ['A', 'B', 'A']
print('All test cases passed!') 


All test cases passed!


## Task 3 - Prediction [2 Points]

### Task 3.1 - Compute Majority Voting [1 Point]

Once we have the `k` nearest neighbors, we need to decide the final label, which is the label that appears the most frequently. If there is a tie, return the label that appears first in the input list.

In [3]:
# TASK 3.1
def knn_majority_vote(neighbors):
    """
    TODO: Return the most common label in neighbors.

    Args:
        neighbors: list of labels

    Returns:
        The label that appears most frequently
        If there's a tie, return the label that appears first
    """
    
    most_common_label = None

    """ YOUR CODE STARTS HERE """
    label_counts = {}
    for label in neighbors:
        label_counts[label] = label_counts.get(label, 0) + 1

    max_count = 0

    for label in neighbors:
        count = label_counts[label]
        if count > max_count:
            max_count = count
            most_common_label = label

    """ YOUR CODE ENDS HERE """

    return most_common_label

# TESTCASES 3.1
assert knn_majority_vote(['A', 'B', 'A']) == 'A'
assert knn_majority_vote(['B', 'B', 'A']) == 'B'
assert knn_majority_vote(['A', 'A', 'A']) == 'A'
assert knn_majority_vote(['A', 'B']) == 'A'
assert knn_majority_vote(['B', 'A']) == 'B'
assert knn_majority_vote(['B', 'A', 'B', 'A']) == 'B'
assert knn_majority_vote(['A', 'B', 'A', 'B']) == 'A'
print('All test cases passed!')


All test cases passed!


### Task 3.2 - Compute KNN Regression Prediction [1 Point]

Now, you will implement K-Nearest Neighbors (KNN) for regression tasks.

In [4]:
# TASK 3.2
def knn_regression(X_train, y_train, x_query, k):
    """
    TODO: Implement the KNN regression algorithm.

    Args:
        X_train (list[list[float]]): Training features
        y_train (list[float]): Target values
        x_query (list[float]): Query point
        k (int): Number of neighbors

    Returns:
        float: Predicted target value by averaging the k nearest neighbors
    """

    prediction = 0

    """ YOUR CODE STARTS HERE """
    # Compute the distance from the query point to each training example
    distances = []
    for i, features in enumerate(X_train):
        dist = euclidean_distance(features, x_query)
        distances.append((dist, i))

    # Sort by distance and select the k nearest neighbors
    distances.sort(key=lambda x: x[0])
    k_nearest = distances[:k]

    # Compute the average of the target values of the k nearest neighbors
    prediction = sum(y_train[i] for _, i in k_nearest) / k

    """ YOUR CODE ENDS HERE """

    return prediction

# TESTCASES 3.2
import math

X_train = [[1], [2], [3], [4], [5]]
y_train = [1.1, 1.9, 3.0, 3.9, 5.1]
x_query = [2.5]
assert math.isclose(knn_regression(X_train, y_train, x_query, 2), 2.45, rel_tol=1e-5)

X_train = [[1], [2], [3]]
y_train = [1, 2, 3]
x_query = [2.1]
assert math.isclose(knn_regression(X_train, y_train, x_query, 1), 2, rel_tol=1e-5)

X_train = [[1], [2], [3]]
y_train = [1, 2, 3]
x_query = [2]
assert math.isclose(knn_regression(X_train, y_train, x_query, 3), 2, rel_tol=1e-5)

X_train = [[1, 2], [2, 3], [3, 4]]
y_train = [10, 20, 30]
x_query = [2, 2.5]
assert math.isclose(knn_regression(X_train, y_train, x_query, 2), 15.0, rel_tol=1e-5)

print('All test cases passed!') 


All test cases passed!


## Task 4 - Wrapping in Classes [4 Points]

### Task 4.1 -  KNN Classifier [2 Points]

Here we combine everything into a reusable class to train your own model and make predictions! 

In [5]:
# TASK 4.1
class KNNClassifier:
    def __init__(self, k=3):
        self.k = k
        self.training_data = []  # Will hold tuples of (feature_vector, label)

    def fit(self, X, y):
        """
        TODO: Store the training data. DO NOT return anything!

        Args:
            X: list of feature vectors
            y: list of labels corresponding to the feature vectors
        """

        """ YOUR CODE STARTS HERE """ 
        # Store the training data
        self.training_data = list(zip(X, y))
        # outputs [([1, 2], 'A'), ([2, 3], 'B'), ([3, 4], 'A'), ([5, 5], 'B')]
        """ YOUR CODE ENDS HERE """


    def predict(self, X_test):
        """
        TODO: Predict the class label for each test point in X_test.

        Args:
            X_test: list of feature vectors to classify

        Returns:
            list of predicted labels 
        """
    
        predictions = []

        """ YOUR CODE STARTS HERE """ 
        # For each test point, find the k nearest neighbors and perform majority voting
        for x in X_test:
            # Compute distances from the test point to all training points
            distances = []
            for (x_train, label) in self.training_data:
                dist = euclidean_distance(x_train, x)
                distances.append((dist, label))

            # Sort by distance and select the k nearest neighbors
            distances.sort(key=lambda x: x[0])
            k_nearest = distances[:self.k]

            # Perform majority voting
            vote = knn_majority_vote([label for _, label in k_nearest])
            predictions.append(vote)
        """ YOUR CODE ENDS HERE """ 
        
        return predictions

# TESTCASES 4.1
knn = KNNClassifier(k=3)
X_train = [[1,2],[2,3],[3,4],[5,5]]
y_train = ['A','B','A','B']
knn.fit(X_train, y_train)

X_test = [[1.5,2.5],[4,4]]
assert knn.predict(X_test) == ['A', 'B']

knn = KNNClassifier(k=1)
X_train = [[1,1],[2,2],[3,3],[4,4]]
y_train = ['A','A','B','B']
knn.fit(X_train, y_train)

X_test = [[1.5,1.5],[3.5,3.5]]
assert knn.predict(X_test) == ['A', 'B']

print('All test cases passed!') 

All test cases passed!


### Task 4.2 - KNN Regressor [2 Points]

Similarly, we do the same for the regressor.

In [6]:
# TASK 4.2
class KNNRegressor:
    def __init__(self, k=3):
        self.k = k
        self.training_data = []  # Will hold tuples of (feature_vector, label)

    def fit(self, X, y):
        """
        TODO: Store the training data. DO NOT return anything!

        Args:
            X: list of feature vectors
            y: list of labels corresponding to the feature vectors
        """

        """ YOUR CODE STARTS HERE """ 
        self.training_data = list(zip(X, y))
        # print(self.training_data)
        # outputs are [([1], 10.0), ([2], 20.0), ([3], 30.0), ([4], 40.0), ([5], 50.0)]
        # [([1, 1], 10.0), ([2, 2], 20.0), ([3, 3], 30.0)]
        """ YOUR CODE ENDS HERE """ 

    def predict(self, X_test):
        """
        TODO: Predict the target value for each test point in X_test

        Args:
            X_test: list of feature vectors to predict

        Returns:
            list of predicted target values 
        """

        predictions = []

        """ YOUR CODE STARTS HERE """ 
        # For each test point, find the k nearest neighbors and predict the target value
        for x in X_test:
            # Compute distances from the test point to all training points
            distances = []
            for (x_train, label) in self.training_data:
                dist = euclidean_distance(x_train, x)
                distances.append((dist, label))

            # Sort by distance and select the k nearest neighbors
            distances.sort(key=lambda x: x[0])
            k_nearest = distances[:self.k]

            # Predict the target value by averaging the k nearest neighbors
            prediction = sum(label for _, label in k_nearest) / self.k
            predictions.append(prediction)
        """ YOUR CODE ENDS HERE """ 

        return predictions

# TESTCASES 4.2
import math

regressor = KNNRegressor(k=3)
X_train = [[1], [2], [3], [4], [5]]
y_train = [10.0, 20.0, 30.0, 40.0, 50.0]
regressor.fit(X_train, y_train)
X_test = [[2.5], [4.5]]
predictions = regressor.predict(X_test)
assert math.isclose(predictions[0], 20.0)
assert math.isclose(predictions[1], 40.0)

regressor = KNNRegressor(k=1)
X_train = [[1, 1], [2, 2], [3, 3]]
y_train = [10.0, 20.0, 30.0]
regressor.fit(X_train, y_train)
X_test = [[1.2, 1.2], [2.8, 2.8], [0.5, 0.5]]
predictions = regressor.predict(X_test)
assert math.isclose(predictions[0], 10.0)
assert math.isclose(predictions[1], 30.0)
assert math.isclose(predictions[2], 10.0)

print('All test cases passed!')     

All test cases passed!


## Task 5 - Practical [2 Points]

Train a KNN classifier on the training dataset using `scikit-learn` and tune its hyperparameters to optimize performance. We recommend that you use PCA as described in the lecture (or any other preprocessing method that you think is suitable) to also boost the model performance. You may find `make_pipeline()` useful in this task.

You will get full marks if your modelling is appropriate and performs well. But remember, you **MUST NOT** use or access X_test and y_test in your code, as this defeats the purpose of a hidden test set. Any model that does so will be given 0 mark.

Make sure that you have installed `scikit-learn` in your python environment. 

**HINT**: Set the `random_state` parameter (if exists) to a certain constant to make your model reproducible (same result on every run)

In [10]:
    # TASK 5.1
    from sklearn.datasets import fetch_lfw_people
    from sklearn.model_selection import train_test_split
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.pipeline import make_pipeline
    from sklearn.decomposition import PCA

    # [Optional] TODO: Add other sklearn imports for your code 
    """ YOUR CODE STARTS HERE """
    from sklearn.metrics import accuracy_score
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import GridSearchCV

    """ YOUR CODE ENDS HERE """

    # Load dataset
    lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
    X = lfw_people.data  # Flattened images
    y = lfw_people.target

    # Split into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=42)

    def train_model(X_train, y_train):
        """
        TODO: Train and return a kNN classifier.
        If using PCA, use a make_pipeline() to combine PCA and kNN.
        When .predict() is called, the model should be able to perform any necessary transformations (like PCA) 
        on the test data automatically.

        Args:
            X_train: Training feature vectors
            y_train: Training labels

        Returns:
            A trained sklearn model, your model will be used to predict the labels of test data
        """

        model = None

        """ YOUR CODE STARTS HERE """ 
        # Create a pipeline with StandardScaler, PCA, and KNeighborsClassifier
        pipeline = make_pipeline(StandardScaler(), PCA(random_state = 42), KNeighborsClassifier(n_neighbors=5))

        param_grid = {
            'kneighborsclassifier__n_neighbors': [3, 5, 7, 9],
            'pca__n_components': [100, 150, 200],
            "kneighborsclassifier__metric": ["euclidean", "manhattan"]
        }

        grid = GridSearchCV(estimator = pipeline, param_grid = param_grid, cv=5, scoring='accuracy')

        grid.fit(X_train, y_train)
        return grid.best_estimator_

    # TESTCASES 5.1
    # Our hidden test cases will use your code to train a model to predict the labels of the test data, not necessarily on the same train-test split.
    # Note: If your model is poorly designed or performs poorly, points may be deducted.

    model = train_model(X_train, y_train)
    # Check if the model can predict
    predictions = model.predict(X_test)
    assert len(predictions) == len(X_test)
    accuracy_score = model.score(X_test, y_test)
    print(f"Model accuracy: {accuracy_score:.2f}")

Model accuracy: 0.67


## END OF ASSIGNMENT